In [32]:
import psycopg2
from psycopg2.extensions import AsIs
import pandas as pd

In [33]:
%run -i ../sql_helpers.py
display(db_credentials)

{'dbname': 'gta',
 'port': 5432,
 'user': 'gta_p8',
 'password': 'r7sdkfdq',
 'host': 'ikgpgis.ethz.ch'}

In [34]:
kirchen_zuerich = pd.read_csv("csv/kirchen-zurich.csv", encoding = "utf-8", sep="	")
restaurant_zurich = pd.read_csv("csv/restaurant-zurich.csv", encoding = "utf-8", sep="	")
kirchen_zuerich.head

<bound method NDFrame.head of           @lat      @lon                                       name  \
0    47.371937  8.520866                      Synagoge Agudas Achim   
1    47.375034  8.527694          Evangelisch Methodistische Kirche   
2    47.402983  8.574152                          Kirchgemeindehaus   
3    47.369365  8.553327     Mission catholique de langue française   
4    47.369483  8.553266                 Couvent St-Nicolas de Myre   
..         ...       ...                                        ...   
115  47.332704  8.516048                            Kirche Leimbach   
116  47.365626  8.553062              Kirche Christi Wissenschafter   
117  47.414579  8.518635  Kirchliches Zentrum Neuaffoltern Glaubten   
118  47.370554  8.505807                        Thomaskirche im Gut   
119  47.360712  8.505901                                Friesenberg   

             addr:street addr:housenumber  addr:postcode addr:city building  \
0           Erikastrasse              

In [38]:
for idx, row in kirchen_zuerich.iterrows():
    #prepare input values
    church_name = row['name']
    address_street = row['addr:street']
    address_number = row['addr:housenumber']
    city_postcode = row['addr:postcode']
    city_name = row['addr:city']
    lon = row["@lon"]   
    lat = row["@lat"]


        
